In [1]:
!pip install -q stanza
!pip install -q pydrive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [40]:
from pathlib import Path
import stanza
from stanza.utils.conll import CoNLL
from io import StringIO
# download and initialize Stanza pipeline
stanza.download("de")
nlp_de = stanza.Pipeline('de', processors="tokenize,mwt,pos,lemma,depparse", tokenize_no_ssplit=True, use_gpu=True)

stanza.download("en")
nlp_en = stanza.Pipeline('en', processors="tokenize,mwt,pos,lemma,depparse", tokenize_no_ssplit=True, use_gpu=True)


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: de (German) ...
INFO:stanza:File exists: /root/stanza_resources/de/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: de (German):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [45]:
folder_name = input("Enter folder name: ")

def create_merged_conllu(df, text_col, lang_code, out_dir="conllu_file", folder_name=folder_name,file_name = ""):
    out_path = Path(out_dir) /  lang_code /folder_name
    out_path.mkdir(parents=True, exist_ok=True)
    merged_file = out_path / f"{file_name}_{lang_code}.conllu"

    nlp = nlp_de if lang_code == 'de' else nlp_en

    sent_counter = 0
    char_offset = 0


    all_text = "\n\n".join(df[text_col].astype(str).tolist())
    doc = nlp(all_text)

    s = StringIO()
    CoNLL.write_doc2conll(doc, s)

    conllu_str = s.getvalue()

    with open(merged_file, "w", encoding="utf-8") as out_f:
      out_f.write(conllu_str)

    return merged_file


Enter folder name: Art


In [46]:
from google.colab import files
import zipfile
import os

# Step 1: Upload ZIP
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]  # Auto get uploaded filename

# Step 2: Extract ZIP into /content/<zip_name_without_extension>
extract_path = os.path.join("/content", os.path.splitext(zip_path)[0])
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Files extracted to:", extract_path)

# Step 3: Recursively search for .xlsx files
excel_files = []
for root, dirs, files_in_dir in os.walk(extract_path):
    for file in files_in_dir:
        if file.lower().endswith(".xlsx"):
            excel_files.append(os.path.join(root, file))


Saving Art_aligned (1).zip to Art_aligned (1) (1).zip
✅ Files extracted to: /content/Art_aligned (1) (1)


In [47]:
import pandas as pd
import glob


dfs = {}
for f in excel_files:
    print(f"\n📥 Reading {os.path.basename(f)} ...")
    df = pd.read_excel(f, engine="openpyxl")
    dfs[os.path.basename(f)] = df

    # Remove extension for clean name
    file_name = os.path.basename(f).rsplit('_', 1)[0]
    print(file_name)

    # Example: merge your German + English cols
    merged_de_file = create_merged_conllu(df, text_col="de", lang_code="de", file_name=file_name)
    merged_en_file = create_merged_conllu(df, text_col="en", lang_code="en", file_name=file_name)



📥 Reading Art16_aligned.xlsx ...
Art16

📥 Reading Art10_aligned.xlsx ...
Art10

📥 Reading Art7_aligned.xlsx ...
Art7

📥 Reading Art13_aligned.xlsx ...
Art13

📥 Reading Art14_aligned.xlsx ...
Art14

📥 Reading Art1_aligned.xlsx ...
Art1

📥 Reading Art8_aligned.xlsx ...
Art8

📥 Reading Art9_aligned.xlsx ...
Art9

📥 Reading Art3_aligned.xlsx ...
Art3

📥 Reading Art11_aligned.xlsx ...
Art11

📥 Reading Art2_aligned.xlsx ...
Art2

📥 Reading Art6_aligned.xlsx ...
Art6

📥 Reading Art15_aligned.xlsx ...
Art15

📥 Reading Art12_aligned.xlsx ...
Art12

📥 Reading Art5_aligned.xlsx ...
Art5

📥 Reading Art4_aligned.xlsx ...
Art4


In [26]:
#!rm -rf /content/3



In [48]:

!zip -r conllu_file.zip conllu_file

from google.colab import files
files.download("conllu_file.zip")


updating: conllu_file/ (stored 0%)
updating: conllu_file/de/ (stored 0%)
updating: conllu_file/de/Art/ (stored 0%)
updating: conllu_file/de/Art/Art12_de.conllu (deflated 81%)
updating: conllu_file/de/Art/Art3_de.conllu (deflated 80%)
updating: conllu_file/de/Art/Art11_de.conllu (deflated 82%)
updating: conllu_file/de/Art/Art5_de.conllu (deflated 81%)
updating: conllu_file/de/Art/Art16_de.conllu (deflated 81%)
updating: conllu_file/de/Art/Art6_de.conllu (deflated 81%)
updating: conllu_file/de/Art/Art10_de.conllu (deflated 80%)
updating: conllu_file/de/Art/Art8_de.conllu (deflated 81%)
updating: conllu_file/de/Art/Art7_de.conllu (deflated 82%)
updating: conllu_file/de/Art/Art14_de.conllu (deflated 81%)
updating: conllu_file/de/Art/Art1_de.conllu (deflated 81%)
updating: conllu_file/de/Art/Art13_de.conllu (deflated 82%)
updating: conllu_file/de/Art/Art15_de.conllu (deflated 81%)
updating: conllu_file/de/Art/Art4_de.conllu (deflated 80%)
updating: conllu_file/de/Art/Art2_de.conllu (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>